In [11]:
!git clone https://github.com/PonyGE/PonyGE2.git

fatal: destination path 'PonyGE2' already exists and is not an empty directory.


In [12]:
!cd PonyGE2/ && git pull

Already up to date.


In [16]:
#pip installs

%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install sewar


In [ ]:
%pip install openpyxl


In [14]:
import shutil

experiment = "regression"

shutil.move(f"./fitness/{experiment}.py", "./PonyGE2/src/fitness/")


'./PonyGE2/src/fitness/regression.py'

In [15]:
!cd ./PonyGE2/src && python ponyge.py --parameters ../../parameters/{experiment}.txt
shutil.move(f"./PonyGE2/src/fitness/{experiment}.py", "./fitness/")


Start:	 2023-02-21 20:29:18.409641 



Traceback (most recent call last):
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\utilities\algorithm\initialise_run.py", line 262, in return_attr_from_module
    module = importlib.import_module(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hsabi\AppData\Local\Programs\Python\Python311\Lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1206, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1178, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1149, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\hsabi\Projects\AIBox\Regresso

'./fitness/regression.py'